Conforme regramento específico, a tarefa será dividida da seguinte forma :

# Conteúdo
1. [Introdução](#1)
2. [Análise exploratória de dados - EDA](#2)
3. [Estratégia de validação](#3)
4. [Treino do modelo](#4)
5. [Submissão](#5)

Formatação realizada com ajuda de [StackOverflow](https://stackoverflow.com/questions/11948245/markdown-to-create-pages-and-table-of-contents).



## 1. Introdução <a name="1"></a>

Este trabalho constitui desafio vivencial da trilha de formação de cientista de dados na [PETROBRAS](https://petrobras.com.br).
Foi escolhida a competição de [Santander Customer Transaction Prediction](https://www.kaggle.com/c/santander-customer-transaction-prediction/overview). Essa é uma competição encerrada em Abril de 2019 que ofereceu US$ 65.000 do 1° ao 5° lugares. Nessa competição, o objetivo é prever quais clientes irão realizar uma transação específica no futuro, independente do valor.

In [ ]:
%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # gráficos

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%time
train_df= pd.read_csv("../input/santander-customer-transaction-prediction/train.csv")
test_df = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')
sample_submission = pd.read_csv('../input/santander-customer-transaction-prediction/sample_submission.csv')
sample_submission.head()

## 2. Análise exploratória de dados - EDA <a name="2"></a>


[For Beginners Santander Prediction LGBM MODEL](https://www.kaggle.com/enesimek/for-beginners-santander-prediction-lgbm-model)



[Santander EDA and Prediction](https://www.kaggle.com/gpreda/santander-eda-and-prediction)

[Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html)

Na descrição dos dados. é informado que os valores disponibilizados para previsão são numéricos e anonimizados, (conforme observado em [Santander Customer Transaction - EDA](https://www.kaggle.com/allunia/santander-customer-transaction-eda)) e tem estrutura similar à disponível para a solução do probelma, o que sugere que os dados são sintéticos.
Por se tratar de informação de transações financeiras, é compreensível que os dados sejam anonimizados, porém o fato de não saber o que cada uma das muitas colunas significa, dificulta a análise de dados e possíveis idéias de combinação de variáveis para melhorar o resultado do modelo.

In [ ]:
%time
train_df.describe()

É possível observar que as médias e desvios padrão estão distribuídos em uma grande variedade de valores, o que leva a entender que esses dados não estão padronizados, o que pode ser um problema para modelos baseados em redes neurais, porém é indiferente para modelos baseados em árvores de decisão, como o que será utilizado.

In [ ]:
%time
train_df.info(verbose = True, show_counts = True)

A partir da função info(), é possível observar que os dados de treino são compostos de um campo de texto "ID_code", um campo inteiro "target" (que pode assumir os valores de 0 ou 1) e todos os demais campos são do tipo número real. A partir dessa mesma função é possível observar que não existem valores faltantes (NaN).

In [ ]:
sns.countplot(x = train_df.target.values)

A partir da contagem da variável "target" é possível observar que existe um desequilíbrio de classes uma vez que existem muito mais transações não realizadas que transações realizadas.

In [ ]:
#sns.pairplot(train_df, hue="target")

In [ ]:
test_df.describe()

In [ ]:
%time
sns.countplot(x = train_df['target'])

## 3. Estratégia de validação <a name="3"></a>
Foi utilizada a divisão dos dados entre treino e teste. Não foi realizada divisão de dados para validação uma vez que os hiperparâmetros ótimos já estavam disponíveis.

## 4. Treino do modelo <a name="4"></a>

O treino do modelo foi realizado com código obtido do notebook [922 in 3 minutes](https://www.kaggle.com/dott1718/922-in-3-minutes/notebook), conforme segue. Assim como a maioria das submissões com melhor classificação no ranking da competição, nesse código é utilizado um modelo do tipo [LightGBM](https://www.kaggle.com/prashant111/lightgbm-classifier-in-python).

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from scipy.special import logit

#train_df = pd.read_csv("../input/train.csv")
#test_df = pd.read_csv("../input/test.csv")
features = [x for x in train_df.columns if x.startswith("var")]

hist_df = pd.DataFrame()
for var in features:
    var_stats = train_df[var].append(test_df[var]).value_counts()
    hist_df[var] = pd.Series(test_df[var]).map(var_stats)
    hist_df[var] = hist_df[var] > 1

ind = hist_df.sum(axis=1) != 200
var_stats = {var:train_df[var].append(test_df[ind][var]).value_counts() for var in features}

pred = 0
for var in features:
    model = lgb.LGBMClassifier(**{
        'learning_rate':0.05, 'max_bin': 165, 'max_depth': 5, 'min_child_samples': 150,
        'min_child_weight': 0.1, 'min_split_gain': 0.0018, 'n_estimators': 41,
        'num_leaves': 6, 'reg_alpha': 2.0, 'reg_lambda': 2.54, 'objective': 'binary', 'n_jobs': -1})
    model = model.fit(np.hstack([train_df[var].values.reshape(-1,1),
                                 train_df[var].map(var_stats[var]).values.reshape(-1,1)]),
                               train_df["target"].values)
    pred += logit(model.predict_proba(np.hstack([test_df[var].values.reshape(-1,1),
                                 test_df[var].map(var_stats[var]).values.reshape(-1,1)]))[:,1])
    

## 5. Submissão <a name="5"></a>
O resultado do modelo é salvo em um arquivo .CSV entitulado "submission".

In [ ]:
%time
pd.DataFrame({"ID_code":test_df["ID_code"], "target":pred}).to_csv("submission.csv", index=False)